In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import time

DC定式化による実験

In [11]:
def dc_formulation(T, C, M, epsilon, xi, t):
    K, J = C.shape
    n = len(xi)

    start = time.time()

    # モデルの定義
    model = gp.Model("dc_formulation")

    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")

    # 目的関数
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + t * gp.quicksum([z[i] * (1-z[i]) for i in range(n)])
    model.setObjective(objective, GRB.MINIMIZE)

    # 制約条件
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])

    # e^{¥top}z<=n*epsilon
    model.addConstr(gp.quicksum(z[i] for i in range(n)) <= n * epsilon)

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])  

    # 時間上限
    model.Params.TimeLimit = 300

    # 最適化
    model.optimize()

    # 結果の表示
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
        optimal_z = np.array([z[i].x for i in range(n)])
        optimal_obj_value = model.objVal
        print("最適解X:", optimal_X)
        print("最適解z:", optimal_z)
        print(optimal_X.shape)
        print("最適解の目的関数値：", optimal_obj_value)
        return optimal_X, optimal_z, optimal_obj_value

        # optimal_zの値をファイルに書き込む
        # listとして保存
        # with open("optimal_z_0621.txt", mode="a") as f:
        #     f.write(str(K)+","+str(J)+"\n")
        #     f.write(str(optimal_z) + "\n")
    else:
        print("最適解は見つかりませんでした。")

    model.close()
    end = time.time()
    print("処理時間：", end-start, "秒")

パラメーターの設定

In [195]:
# シードの設定（再現性のため）
np.random.seed(42)

# 供給者数
K = 4
# 消費者数
J = 8
# シナリオ数
n = 10

T = np.ones((1, K))
# print("T", T)
C = np.random.uniform(10, 50, size=(K, J))
# print("c", C)
M = np.array([100] * K)

# 消費者の需要を表すランダムベクトルの生成
xi = []
for _ in range(n):
    mean = np.random.uniform(0, 10, J)  # 平均ベクトルをランダムに生成
    variance = np.random.uniform(1, 5, J)  # 分散ベクトルをランダムに生成
    consumer_demand = np.abs(np.random.normal(mean, np.sqrt(variance), J))  # 正規分布に従うランダムベクトルを生成
    xi.append(consumer_demand)
xi = np.array(xi)

z_t_minus_1 = [0.5] * n

In [24]:
dc_formulation(T, C, M, 0.3, xi, 1000)

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xbb73f3ec
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+03]
  QObjective range [2e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]

Continuous model is non-convex -- solving as a MIP

Presolve removed 4 rows and 24 columns
Presolve time: 0.00s
Presolved: 92 rows, 29 columns, 201 nonzeros
Presolved model has 10 bilinear constraint(s)
Variable types: 29 continuous, 0 integer (0 binary)
Found heuristic solution: objective 1535.0880747

Root relaxation: objective 9.459259e+02, 44 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds    

(array([[ 0.        ,  0.        ,  9.30749305,  0.        ],
        [ 0.        ,  0.        , 11.28862303,  0.        ],
        [ 0.        , 12.52853261,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  8.45300391,  0.        ],
        [ 7.56286411,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  7.65457003],
        [ 8.59222919,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.77253416]]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 1., 1.]),
 1358.6461377275064)

PMIP

In [70]:
def solv_pmip(T, C, M, epsilon, xi):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    
    K, J = C.shape
    n = len(xi)

    start = time.time()
    # モデルの定義
    model = gp.Model("PMIP")

    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, vtype=GRB.BINARY, name="z")

    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J))
    model.setObjective(objective, sense=GRB.MINIMIZE)

    # 制約条件の定義
    # rhs = []
    # for i in range(n):
    #     rhs.append()
    # print(len(rhs))
    start_seiyaku = time.time()
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])    
    end_seiyaku = time.time()
    seiyaku_time = end_seiyaku-start_seiyaku
    print("制約条件追加の処理時間：", seiyaku_time)

    model.addConstr(gp.quicksum(z[i] for i in range(n)) <= int(n * epsilon))
    
    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])

    # 時間上限
    model.Params.TimeLimit = 3000
    # 最適化の実行
    model.optimize()

    # 結果の表示
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
        optimal_z = np.array([z[i].x for i in range(n)])
        optimal_obj_value = model.objVal
        print("最適解X:", optimal_X)
        print("最適解z:", optimal_z)
        print(optimal_X.shape)
        print("最適解の目的関数値：", optimal_obj_value)
        return optimal_X, optimal_z, optimal_obj_value

        # optimal_zの値をファイルに書き込む
        # listとして保存
        # with open("optimal_z_0621.txt", mode="a") as f:
        #     f.write(str(K)+","+str(J)+"\n")
        #     f.write(str(optimal_z) + "\n")
    else:
        print("最適解は見つかりませんでした。")

    model.close()
    end = time.time()
    print("処理時間：", end-start, "秒")

In [54]:
solv_pmip(T=T, C=C, M=M, epsilon=0.3, xi=xi)

制約条件追加の処理時間： 0.0021429061889648438
Set parameter TimeLimit to value 3000
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xc9a7384b
Variable types: 32 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 4 rows and 24 columns
Presolve time: 0.00s
Presolved: 81 rows, 18 columns, 242 nonzeros
Variable types: 8 continuous, 10 integer (10 binary)
Found heuristic solution: objective 1445.9956374
Found heuristic solution: objective 1413.9847239
Found heuristic solution: objective 1358.6461377

Root relaxation: objective 1.115922e+03, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Obj

(array([[ 0.        ,  0.        ,  9.30749305,  0.        ],
        [ 0.        ,  0.        , 11.28862303,  0.        ],
        [ 0.        , 12.52853261,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  8.45300391,  0.        ],
        [ 7.56286411,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  7.65457003],
        [ 8.59222919,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.77253416]]),
 array([-0., -0., -0., -0.,  1., -0., -0., -0.,  1.,  1.]),
 1358.6461377275068)

In [188]:
# シードの設定（再現性のため）
np.random.seed(42)

# 供給者数
K = 40
# 消費者数
J = 100
# シナリオ数
n = 1000

T = np.ones((1, K))
# print("T", T)
C = np.random.uniform(10, 50, size=(K, J))
# print("c", C)
M = np.array([100] * K)

# 消費者の需要を表すランダムベクトルの生成
xi = []
for _ in range(n):
    mean = np.random.uniform(0, 10, J)  # 平均ベクトルをランダムに生成
    variance = np.random.uniform(1, 5, J)  # 分散ベクトルをランダムに生成
    consumer_demand = np.abs(np.random.normal(mean, np.sqrt(variance), J))  # 正規分布に従うランダムベクトルを生成
    xi.append(consumer_demand)
xi = np.array(xi)

z_t_minus_1 = [0.5] * n

In [55]:
dc_formulation(T, C, M, 0.3, xi, 1000)

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xbb73f3ec
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+03]
  QObjective range [2e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]

Continuous model is non-convex -- solving as a MIP

Presolve removed 4 rows and 24 columns
Presolve time: 0.00s
Presolved: 92 rows, 29 columns, 201 nonzeros
Presolved model has 10 bilinear constraint(s)
Variable types: 29 continuous, 0 integer (0 binary)
Found heuristic solution: objective 1535.0880747

Root relaxation: objective 9.459259e+02, 44 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds    

(array([[ 0.        ,  0.        ,  9.30749305,  0.        ],
        [ 0.        ,  0.        , 11.28862303,  0.        ],
        [ 0.        , 12.52853261,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  8.45300391,  0.        ],
        [ 7.56286411,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  7.65457003],
        [ 8.59222919,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.77253416]]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 1., 1.]),
 1358.6461377275064)

DCA

In [8]:
# 劣勾配計算
def process_vector(w, K):
    w = np.asarray(w, dtype=float)
    # print("w", w)
    # wを減少順に並べ替える
    sorted_indices = np.argsort(-np.abs(w))
    sorted_w = np.abs(w[sorted_indices])
    # print("sorted indices", sorted_indices)
    # print("sorted_w", sorted_w)

    # w(i)をK番目まで1、それ以外を0にする
    s = np.zeros_like(w)
    s[sorted_indices[:K]] = np.sign(w[sorted_indices[:K]])
    # print("s", s)

    return s

In [30]:
# 凸最適化問題を解く
def solve_main(T, C, M, xi, rho, s_z_t_minus_1):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)
    # モデルの作成
    model = gp.Model("DCA")
    
    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")
    
    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z[i] for i in range(n))- gp.quicksum(z[i] * rho * s_z_t_minus_1[i] for i in range(n))
    model.setObjective(objective, sense=GRB.MINIMIZE)
    
    # 制約条件の定義
    # rhs = []
    # for i in range(n):
    #     rhs.append((1-z[i]) * xi[i])
    # start_seiyaku = time.time()
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])    
    # end_seiyaku = time.time()
    # seiyaku_time = end_seiyaku-start_seiyaku
    # print("制約条件追加の処理時間：", seiyaku_time, "秒")

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])
        
    # 時間上限
    model.Params.TimeLimit = 3600
    
    # 最適化の実行
    model.optimize()
    
    # 結果の表示
    # if model.status == GRB.OPTIMAL:
    #     optimal_X = np.array([[X[k, j].x for j in range(J)] for k in range(K)]) 
    #     optimal_z = np.array([z[i].x for i in range(n)])
    #     optimal_obj_value = model.objVal
    #     return optimal_X, optimal_z, optimal_obj_value, seiyaku_time
    # else:
    #     print("最適解が見つかりませんでした。")
    #     return None, None, None, seiyaku_time

    optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
    optimal_z = np.array([z[i].x for i in range(n)])
    optimal_obj_value = model.objVal

    return optimal_X, optimal_z, optimal_obj_value

In [193]:
def dca(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon=0.001, max_iteration=100):
    # 初期値
    f_t_minus_1 = np.inf
    start = time.time()
    iteration = 0
    for iteration in range(max_iteration):
    # while True:
        print("-----反復回数：", iteration+1, "回-----")

        # 劣勾配の計算
        s_t_minus_1 = process_vector(z_t_minus_1, int(len(xi) * epsilon))
        print("s_t_minus_1:", s_t_minus_1)
        
        # 最適化問題を解く
        X_t, z_t, f_t = solve_main(T, C, M, xi, rho, s_t_minus_1)
        print("z_t:", z_t)

        # 収束判定
        if np.abs(f_t_minus_1 - f_t) < varepsilon:
            print("解が収束しました。")
            break

        # 更新
        f_t_minus_1 = f_t
        z_t_minus_1 = z_t
        # iteration += 1
    

        # if iteration == max_iteration:
        #     print("最大反復回数に達しました。")
        #     break

    print("最適解x:", X_t) 
    print("最適解z:", z_t)
    print("最適値f：", f_t)
    end = time.time()

In [196]:
dca(T, C, M, xi, 1000, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x71fb86e8
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.00s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
       8    1.5941748e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.594174818e+03


z_t: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
-----反復回数： 2 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x71fb86e8
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.00s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
       8    1.5941748e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.594174818e+03
z_t: [1. 1. 1. 0. 0. 0. 0. 0. 0.

In [197]:
dca(T, C, M, xi, 500, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x2677b47a
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.00s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
       8    1.5941748e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.594174818e+03
z_t: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
-----反復回数： 2 回-----
s_t_minus_1:

In [198]:
dca(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xb0cbe4b4
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.00s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
      17    1.3935349e+03   0.000000e+00   0.000000e+00      0s

Solved in 17 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.393534943e+03
z_t: [1.         1.         1.         0.33695458 0.39635223 0.
 0.3

In [52]:
dca(T, C, M, xi, 300, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x40738fdd
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.00s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
      11    1.5752586e+03   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.575258577e+03
z_t: [1.         1.         1.         0.         0.         0.
 0. 

In [199]:
dca(T, C, M, xi, 100, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xd249fab6
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.02s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
       8    7.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.04 seconds (0.00 work units)
Optimal objective  7.000000000e+02
z_t: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
-----反復回数： 2 回-----
s_t_minus_1:

In [189]:
dca(T, C, M, xi, 1000, 0.1, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.

In [192]:
dca(T, C, M, xi, 100, 0.1, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.

In [57]:
solve_main(T, C, M, xi, 300, [1,1,1,0,0,0,0,0,0,0])

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)



CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x40738fdd
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.04s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
      11    1.5752586e+03   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.05 seconds (0.00 work units)
Optimal objective  1.575258577e+03


(array([[ 0.        ,  0.        , 12.61038026,  0.        ],
        [ 0.        ,  0.        ,  7.57892622,  0.        ],
        [ 0.        , 12.52853261,  0.        ,  0.        ],
        [ 0.        ,  0.        , 11.02253105,  0.        ],
        [10.49313805,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.25509261],
        [ 8.59222919,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.77253416]]),
 array([1.        , 1.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.32862261, 0.12523051, 0.        ]),
 1575.2585774717788)

In [58]:
solve_main(T, C, M, xi, 300, [1, 1, 1, 0, 0, 0, 0, 0.32862261, 0.12523051, 0])

Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x49dff79f
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.01s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
      12    1.5381559e+03   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.538155927e+03


(array([[ 0.        ,  0.        , 12.61038026,  0.        ],
        [ 0.        ,  0.        ,  7.57892622,  0.        ],
        [ 0.        , 12.52853261,  0.        ,  0.        ],
        [ 0.        ,  0.        , 11.02253105,  0.        ],
        [10.49313805,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.25509261],
        [ 8.59222919,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.77253416]]),
 array([1.        , 1.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.32862261, 0.12523051, 0.        ]),
 1538.1559267625146)

In [61]:
dca(T, C, M, xi, 300, 0.3, np.random.rand(10))

-----反復回数： 1 回-----
s_t_minus_1: [0. 0. 0. 0. 0. 1. 0. 0. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xbb7e9469
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.00s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.008346e+01   0.000000e+00      0s
      12    1.4275996e+03   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.427599599e+03


z_t: [0.         0.         0.         0.19618556 0.23311589 1.
 0.         0.1293242  1.         1.        ]
-----反復回数： 2 回-----
s_t_minus_1: [0. 0. 0. 0. 0. 1. 0. 0. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xbb7e9469
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.00s
Presolved: 60 rows, 39 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.008346e+01   0.000000e+00      0s
      12    1.4275996e+03   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.00 seconds (0.00 work 

DCA改善

In [62]:
def solve_X(T, C, M, xi, epsilon, lambda_, z_0):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)

    # モデルの作成
    model = gp.Model('solve_X')

    # 変数の追加
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name='X')
    # z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name='z')

    # 目的関数の追加
    # zを大きい順に並べ, pからnまでのzを足し合わせる
    # z = np.array(z_0)
    # z_sort = np.sort(z)
    # T_K = gp.quicksum(z_sort[i] for i in range(p+1, n))
    objective  = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J))
    model.setObjective(objective, GRB.MINIMIZE)

    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z_0[i]) * xi[i])
    
    # for i in range(n-1):
    #     model.addConstr(z[i] >= z[i+1])

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])

    # 時間上限
    model.Params.TimeLimit = 3600

    # 最適化の実行
    model.optimize()

    # 結果の表示
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)]) 
        # optimal_z = np.array([z[i].x for i in range(n)])
        optimal_obj_value = model.objVal
        return optimal_X, optimal_obj_value
    else:
        print("最適解が見つかりませんでした。")
        return None, None

In [63]:
def dca_k(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon=0.001, max_iteration=100):
    # 初期値
    f_t_minus_1 = np.inf
    start = time.time()
    iteration = 0
    for iteration in range(max_iteration):
    # while True:
        print("-----反復回数：", iteration+1, "回-----")

        # 劣勾配の計算
        s_t_minus_1 = process_vector(z_t_minus_1, int(len(xi) * epsilon))
        print("s_t_minus_1:", s_t_minus_1)
        
        # 最適化問題を解く
        X_t, z_t, f_t = solve_main(T, C, M, xi, rho, s_t_minus_1)
        print("z_t:", z_t)

        # zをK番目まで1、それ以外を0にする
        z_hat = process_vector(z_t, int(len(xi) * epsilon))

        # Xを求める
        X_t_hat, f_t_hat = solve_X(T, C, M, xi, epsilon, rho, z_hat)

        # 収束判定
        if f_t_minus_1  < f_t_hat:
            print("解が悪化しました。")
            break

        # 更新
        f_t_minus_1 = f_t
        z_t_minus_1 = z_t
        # iteration += 1
    

        # if iteration == max_iteration:
        #     print("最大反復回数に達しました。")
        #     break

    print("最適解x:", X_t) 
    print("最適解z:", z_t)
    print("最適値f：", f_t)
    print("最適値f_hat：", f_t_hat) 
    end = time.time()

In [65]:
dca_k(T, C, M, xi, 300, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x40738fdd
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 24 rows and 3 columns
Presolve time: 0.00s
Presolved: 60 rows, 39 columns, 312 nonzeros



Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.229283e+01   0.000000e+00      0s
      11    1.5752586e+03   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.575258577e+03
z_t: [1.         1.         1.         0.         0.         0.
 0.         0.32862261 0.12523051 0.        ]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 32 columns and 352 nonzeros
Model fingerprint: 0xd8f6677f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 72 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 32 columns, 64 nonzeros

Iteration    Objective  

DC表現を制約に残す

In [ ]:
def solve_X(T, C, M, xi, epsilon, lambda_, z_0):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)

    # モデルの作成
    model = gp.Model('solve_X')

    # 変数の追加
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name='X')
    # z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name='z')

    # 目的関数の追加
    # zを大きい順に並べ, pからnまでのzを足し合わせる
    # z = np.array(z_0)
    # z_sort = np.sort(z)
    # T_K = gp.quicksum(z_sort[i] for i in range(p+1, n))
    objective  = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J))
    model.setObjective(objective, GRB.MINIMIZE)

    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z_0[i]) * xi[i])
    
    # for i in range(n-1):
    #     model.addConstr(z[i] >= z[i+1])

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])

    # 時間上限
    model.Params.TimeLimit = 3600

    # 最適化の実行
    model.optimize()

    # 結果の表示
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)]) 
        # optimal_z = np.array([z[i].x for i in range(n)])
        optimal_obj_value = model.objVal
        return optimal_X, optimal_obj_value
    else:
        print("最適解が見つかりませんでした。")
        return None, None

In [66]:
def dc_formulation_2(T, C, M, epsilon, xi, t):
    K, J = C.shape
    n = len(xi)

    start = time.time()

    # モデルの定義
    model = gp.Model("dc_formulation")

    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")

    # 目的関数
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + t * gp.quicksum(z[i] * (1-z[i]) for i in range(n))
    model.setObjective(objective, GRB.MINIMIZE)

    # 制約条件
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])

    # e^{¥top}z<=n*epsilon
    model.addConstr(gp.quicksum(z[i] for i in range(n)) <= n * epsilon)

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])  

    # 時間上限
    model.Params.TimeLimit = 300

    # 最適化
    model.optimize()

    # 結果の表示
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
        optimal_z = np.array([z[i].x for i in range(n)])
        optimal_obj_value = model.objVal
        print("最適解X:", optimal_X)
        print("最適解z:", optimal_z)
        print(optimal_X.shape)
        print("最適解の目的関数値：", optimal_obj_value)
        return optimal_X, optimal_z, optimal_obj_value

        # optimal_zの値をファイルに書き込む
        # listとして保存
        # with open("optimal_z_0621.txt", mode="a") as f:
        #     f.write(str(K)+","+str(J)+"\n")
        #     f.write(str(optimal_z) + "\n")
    else:
        print("最適解は見つかりませんでした。")

    model.close()
    end = time.time()
    print("処理時間：", end-start, "秒")

In [67]:
dc_formulation_2(T, C, M, 0.3, xi, 1000)

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xbb73f3ec
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+03]
  QObjective range [2e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]

Continuous model is non-convex -- solving as a MIP

Presolve removed 4 rows and 24 columns
Presolve time: 0.00s
Presolved: 92 rows, 29 columns, 201 nonzeros
Presolved model has 10 bilinear constraint(s)
Variable types: 29 continuous, 0 integer (0 binary)
Found heuristic solution: objective 1535.0880747

Root relaxation: objective 9.459259e+02, 44 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds    

(array([[ 0.        ,  0.        ,  9.30749305,  0.        ],
        [ 0.        ,  0.        , 11.28862303,  0.        ],
        [ 0.        , 12.52853261,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  8.45300391,  0.        ],
        [ 7.56286411,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  7.65457003],
        [ 8.59222919,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.77253416]]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 1., 1.]),
 1358.6461377275064)

## 6/26
DC表現を制約に残す

In [114]:
def dc_constraint(T, C, M, epsilon, xi, rho=100):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    
    K, J = C.shape
    n = len(xi)
    p = int(n * epsilon)

    start = time.time()
    # モデルの定義
    model = gp.Model("PMIP")

    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")
    w = model.addVars(n, lb=0.0, vtype=GRB.CONTINUOUS, name="w")
    d = model.addVar(vtype=GRB.CONTINUOUS, name="d")

    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J))
    model.setObjective(objective, sense=GRB.MINIMIZE)

    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])    
    
    # ¥Sigma s_i <= n * epsilon をDC表現
    # ¥sum|z_i| - p * d + ¥sum w_i <= rho
    model.addConstr(p * d + gp.quicksum(w[i] for i in range(n)) >= gp.quicksum(z[i] for i in range(n)) - rho)
    
    # w_i >= |z_i| -d
    for i in range(n):
        model.addConstr(w[i] >= z[i] - d)
        model.addConstr(w[i] >= -z[i] - d)
    
    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])

    # 時間上限
    model.Params.TimeLimit = 3000
    # 最適化の実行
    model.optimize()

    # 結果の表示
    if model.status == GRB.OPTIMAL:
        optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
        optimal_z = np.array([z[i].x for i in range(n)])
        optimal_w = np.array([w[i].x for i in range(n)])
        optimal_d = d.x
        optimal_obj_value = model.objVal
        print("最適解X:", optimal_X)
        print("最適解z:", optimal_z)
        print(optimal_X.shape)
        print("最適解の目的関数値：", optimal_obj_value)
        return optimal_X, optimal_z, optimal_obj_value, optimal_w, optimal_d    

        # optimal_zの値をファイルに書き込む
        # listとして保存
        # with open("optimal_z_0621.txt", mode="a") as f:
        #     f.write(str(K)+","+str(J)+"\n")
        #     f.write(str(optimal_z) + "\n")
    else:
        print("最適解は見つかりませんでした。")

    model.close()
    end = time.time()
    print("処理時間：", end-start, "秒")

In [3]:
# シードの設定（再現性のため）
np.random.seed(42)

# 供給者数
K = 4
# 消費者数
J = 8
# シナリオ数
n = 10

T = np.ones((1, K))
# print("T", T)
C = np.random.uniform(10, 50, size=(K, J))
# print("c", C)
M = np.array([100] * K)

# 消費者の需要を表すランダムベクトルの生成
xi = []
for _ in range(n):
    mean = np.random.uniform(0, 10, J)  # 平均ベクトルをランダムに生成
    variance = np.random.uniform(1, 5, J)  # 分散ベクトルをランダムに生成
    consumer_demand = np.abs(np.random.normal(mean, np.sqrt(variance), J))  # 正規分布に従うランダムベクトルを生成
    xi.append(consumer_demand)
xi = np.array(xi)

z_t_minus_1 = [0.5] * n

In [115]:
dc_constraint(T, C, M, 0.3, xi, rho=1e-5)

Set parameter TimeLimit to value 3000
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 105 rows, 53 columns and 512 nonzeros
Model fingerprint: 0x37f592b7
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 105 rows and 53 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00
最適解X: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
最適解z: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(8, 4)
最適解の目的関数値： 0.0


(array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 0.0,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 0.0)

0-1制約をDC分解→DCアルゴリズムで解く

In [160]:
def subgradient(z_t_minus_1):
    # z_t_minus_1の各要素を2倍したnp.arrayを返す
    return 2 * np.array(z_t_minus_1)


# 凸最適化問題は最大Kノルムの設定に¥sum z_i <n*epsilonを追加したもの
def solv_main(T, C, M, xi, rho, epsilon, s_z_t_minus_1):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)
    # モデルの作成
    model = gp.Model("DCA")
    
    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")
    
    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z[i] for i in range(n))- gp.quicksum(z[i] * rho * s_z_t_minus_1[i] for i in range(n))
    model.setObjective(objective, sense=GRB.MINIMIZE)
    
    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])   

    model.addConstr(gp.quicksum(z[i] for i in range(n)) <= int(n * epsilon))

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])
        
    # 時間上限
    model.Params.TimeLimit = 3600
    
    # 最適化の実行
    model.optimize()
    
    # 結果の表示
    # if model.status == GRB.OPTIMAL:
    #     optimal_X = np.array([[X[k, j].x for j in range(J)] for k in range(K)]) 
    #     optimal_z = np.array([z[i].x for i in range(n)])
    #     optimal_obj_value = model.objVal
    #     return optimal_X, optimal_z, optimal_obj_value, seiyaku_time
    # else:
    #     print("最適解が見つかりませんでした。")
    #     return None, None, None, seiyaku_time

    optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
    optimal_z = np.array([z[i].x for i in range(n)])
    optimal_obj_value = model.objVal

    return optimal_X, optimal_z, optimal_obj_value

def dca_92(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon=0.001, max_iteration=100):
    # 初期値
    start = time.time()
    iteration = 0
    f_t_minus_1 = np.inf
    for iteration in range(max_iteration):
    # while True:
        print("-----反復回数：", iteration+1, "回-----")

        # 劣勾配の計算
        s_t_minus_1 = subgradient(z_t_minus_1)
        print("s_t_minus_1:", s_t_minus_1)
        
        # 最適化問題を解く
        X_t, z_t, _ = solv_main(T, C, M, xi, rho, epsilon, s_t_minus_1)
        print("z_t:", z_t)
        f_t = np.sum(np.transpose(C) * X_t)
        print("最適値：",f_t)

        # zをK番目まで1、それ以外を0にする
        # z_hat = process_vector(z_t, int(len(xi) * epsilon))

        # Xを求める
        # X_t_hat, f_t_hat = solve_X(T, C, M, xi, epsilon, rho, z_hat)

        # 収束判定
        if np.abs(f_t_minus_1 - f_t) < varepsilon:
            print("解が収束しました。")
            break

        # 更新
        f_t_minus_1 = f_t
        z_t_minus_1 = z_t
        # iteration += 1
    

        # if iteration == max_iteration:
        #     print("最大反復回数に達しました。")
        #     break

    print("最適解x:", X_t) 
    print("最適解z:", z_t)
    print("最適値f：", f_t)
    end = time.time()
    print("処理時間：", end-start, "秒")



In [149]:
dca_92(T, C, M, xi, 500, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.00 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.43894892]
最適値： 945.9259374857263
-----反復回数： 2 回-----
s_t_minus_1: [0.82770182 0.39087854 0.10864935 0.47970504

In [156]:
dca_92(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:


  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.01s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.43894892]
最適値： 945.9259374857263
-----反復回数： 2 回-----
s_t_minus_1: [0.82770182 0.39087854 0.10864935 0.47970504 0.79270445 0.
 0.59925992 0.97930834 0.94389469 0.87789785]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a mod

In [182]:
dca_92(T, C, M, xi, 1000, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.01s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

In [150]:
dca_92(T, C, M, xi, 500, 0.3, [0, 0, 0, 0, 1, 0, 0, 0, 1, 1])

-----反復回数： 1 回-----
s_t_minus_1: [0 0 0 0 2 0 0 0 2 2]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0x1dc97b89
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.5000000e+03   1.990822e+02   0.000000e+00      0s
      14   -1.4135386e+02   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.413538623e+02
z_t: [0. 0. 0. 0. 1. 0. 0. 0. 1. 1.]
最適値： 1358.6461377275068
-----反復回数： 2 回-----
s_t_minus_1: [0. 0. 0. 0. 2. 0. 0. 0

In [125]:
solv_pmip(T=T, C=C, M=M, epsilon=0.3, xi=xi)

制約条件追加の処理時間： 0.002235889434814453
Set parameter TimeLimit to value 3000
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xc9a7384b
Variable types: 32 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 4 rows and 24 columns
Presolve time: 0.00s
Presolved: 81 rows, 18 columns, 242 nonzeros
Variable types: 8 continuous, 10 integer (10 binary)
Found heuristic solution: objective 1445.9956374
Found heuristic solution: objective 1413.9847239
Found heuristic solution: objective 1358.6461377

Root relaxation: objective 1.115922e+03, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Obje

(array([[ 0.        ,  0.        ,  9.30749305,  0.        ],
        [ 0.        ,  0.        , 11.28862303,  0.        ],
        [ 0.        , 12.52853261,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  8.45300391,  0.        ],
        [ 7.56286411,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  7.65457003],
        [ 8.59222919,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  9.77253416]]),
 array([-0., -0., -0., -0.,  1., -0., -0., -0.,  1.,  1.]),
 1358.6461377275068)

In [266]:
# シードの設定（再現性のため）
np.random.seed(42)

# 供給者数
K = 40
# 消費者数
J = 100
# シナリオ数
n = 1000

T = np.ones((1, K))
# print("T", T)
C = np.random.uniform(10, 50, size=(K, J))
# print("c", C)
M = np.array([100] * K)

# 消費者の需要を表すランダムベクトルの生成
xi = []
for _ in range(n):
    mean = np.random.uniform(0, 10, J)  # 平均ベクトルをランダムに生成
    variance = np.random.uniform(1, 5, J)  # 分散ベクトルをランダムに生成
    consumer_demand = np.abs(np.random.normal(mean, np.sqrt(variance), J))  # 正規分布に従うランダムベクトルを生成
    xi.append(consumer_demand)
xi = np.array(xi)

z_t_minus_1 = [0.5] * n

In [153]:
dca_92(T, C, M, xi, 200, 0.1, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.

コスト行列にノイズを加える（正規乱数）

In [166]:
# 凸最適化問題は最大Kノルムの設定に¥sum z_i <n*epsilonを追加したもの
def solv_main_noise(T, C, M, xi, rho, epsilon, s_z_t_minus_1):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)
    # モデルの作成
    model = gp.Model("DCA")
    
    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")

    # C[k, j]にexp(si)(siは正規乱数）をかける
    np.random.seed(42)
    C_noise = C * np.exp(np.random.normal(0, 1, (K, J)))
    
    # 目的関数の定義
    objective = gp.quicksum(C_noise[k, j] * X[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z[i] for i in range(n))- gp.quicksum(z[i] * rho * s_z_t_minus_1[i] for i in range(n))
    model.setObjective(objective, sense=GRB.MINIMIZE)
    
    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])   

    model.addConstr(gp.quicksum(z[i] for i in range(n)) <= int(n * epsilon))

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])
        
    # 時間上限
    model.Params.TimeLimit = 3600
    
    # 最適化の実行
    model.optimize()
    
    # 結果の表示
    # if model.status == GRB.OPTIMAL:
    #     optimal_X = np.array([[X[k, j].x for j in range(J)] for k in range(K)]) 
    #     optimal_z = np.array([z[i].x for i in range(n)])
    #     optimal_obj_value = model.objVal
    #     return optimal_X, optimal_z, optimal_obj_value, seiyaku_time
    # else:
    #     print("最適解が見つかりませんでした。")
    #     return None, None, None, seiyaku_time

    optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
    optimal_z = np.array([z[i].x for i in range(n)])
    optimal_obj_value = model.objVal

    return optimal_X, optimal_z, optimal_obj_value

def dca_92_noise(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon=0.001, max_iteration=100):
    # 初期値
    start = time.time()
    iteration = 0
    f_t_minus_1 = np.inf
    for iteration in range(max_iteration):
    # while True:
        print("-----反復回数：", iteration+1, "回-----")

        # 劣勾配の計算
        s_t_minus_1 = subgradient(z_t_minus_1)
        print("s_t_minus_1:", s_t_minus_1)
        
        # 最適化問題を解く
        X_t, z_t, _ = solv_main_noise(T, C, M, xi, rho, epsilon, s_t_minus_1)
        print("z_t:", z_t)
        f_t = np.sum(np.transpose(C) * X_t)
        print("最適値：",f_t)

        # zをK番目まで1、それ以外を0にする
        # z_hat = process_vector(z_t, int(len(xi) * epsilon))

        # Xを求める
        # X_t_hat, f_t_hat = solve_X(T, C, M, xi, epsilon, rho, z_hat)

        # 収束判定
        if np.abs(f_t_minus_1 - f_t) < varepsilon:
            print("解が収束しました。")
            break

        # 更新
        f_t_minus_1 = f_t
        z_t_minus_1 = z_t
        # iteration += 1
    

        # if iteration == max_iteration:
        #     print("最大反復回数に達しました。")
        #     break

    print("最適解x:", X_t) 
    print("最適解z:", z_t)
    print("最適値f：", f_t)
    end = time.time()
    print("処理時間：", end-start, "秒")


In [157]:
np.random.normal(0, 1, (K, J))

array([[-0.71530371,  0.67959775, -0.73036663,  0.21645859,  0.04557184,
        -0.65160035,  2.14394409,  0.63391902],
       [-2.02514259,  0.18645431, -0.66178646,  0.85243333, -0.79252074,
        -0.11473644,  0.50498728,  0.86575519],
       [-1.20029641, -0.33450124, -0.47494531, -0.65332923,  1.76545424,
         0.40498171, -1.26088395,  0.91786195],
       [ 2.1221562 ,  1.03246526, -1.51936997, -0.48423407,  1.26691115,
        -0.70766947,  0.44381943,  0.77463405]])

In [168]:
dca_92_noise(T, C, M, xi, 500, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0x4f90ace7
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      32    5.2807008e+02   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.280700818e+02


z_t: [0.48872623 0.06461019 0.         0.33695458 0.36855324 0.
 0.33196067 0.5548463  0.44762965 0.40671913]
最適値： 1263.332113550114
-----反復回数： 2 回-----
s_t_minus_1: [0.97745246 0.12922039 0.         0.67390916 0.73710649 0.
 0.66392134 1.1096926  0.89525929 0.81343827]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0x873559f2
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [3e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -5.4846300e+01   2.531511e+02   0.000000e+00      0s
      23    7.0661409e+02   0.000000e+00   0.000000e+00     

In [187]:
dca_92_noise(T, C, M, xi, 500, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0x4f90ace7
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      32    5.2807008e+02   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.280700818e+02
z_t: [0.48872623 0.06461019 0.         0.33695458 0.36855324 0.
 0.33196067 0.5548463  0.44762965 0.4067191

In [167]:
dca_92_noise(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0x4f90ace7
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      32    5.2807008e+02   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.280700818e+02
z_t: [0.48872623 0.06461019 0.         0.33695458 0.36855324 0.
 0.33196067 0.5548463  0.44762965 0.4067191

Optimal objective  6.112623822e+02
z_t: [0.48872623 0.         0.         0.33695458 0.40301625 0.
 0.33196067 0.5548463  0.47777683 0.40671913]
最適値： 1290.564340930703
-----反復回数： 3 回-----
s_t_minus_1: [0.97745246 0.         0.         0.67390916 0.8060325  0.
 0.66392134 1.1096926  0.95555367 0.81343827]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0x65bf39d0
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.1938520e+01   2.531511e+02   0.000000e+00      0s
      24    5.9994527e+02

In [169]:
dca_92_noise(T, C, M, xi, 1000, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0x4f90ace7
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      32    5.2807008e+02   0.000000e+00   0.000000e+00      0s

Solved in 32 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.280700818e+02
z_t: [0.48872623 0.06461019 0.         0.33695458 0.36855324 0.
 0.33196067 0.5548463  0.44762965 0.4067191

DCAに正則化項を入れる

In [2]:
# 凸最適化問題を解く
def solve_main_(T, C, M, xi, rho, s_z_t_minus_1, epsilon):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)
    # モデルの作成
    model = gp.Model("DCA")
    
    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")

    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z[i] for i in range(n))- gp.quicksum(z[i] * rho * s_z_t_minus_1[i] for i in range(n)) + 1e-6 * gp.quicksum(z[i] * z[i] for i in range(n))
    model.setObjective(objective, sense=GRB.MINIMIZE)
    
    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])    

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])
        
    # 時間上限
    model.Params.TimeLimit = 3600
    
    # 最適化の実行
    model.optimize()

    optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
    optimal_z = np.array([z[i].x for i in range(n)])
    optimal_obj_value = model.objVal

    return optimal_X, optimal_z, optimal_obj_value

In [6]:
def dca_ridge(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon=0.001, max_iteration=100):
    # 初期値
    f_t_minus_1 = np.inf
    start = time.time()
    iteration = 0
    for iteration in range(max_iteration):
    # while True:
        print("-----反復回数：", iteration+1, "回-----")

        # 劣勾配の計算
        s_t_minus_1 = process_vector(z_t_minus_1, int(len(xi) * epsilon))
        print("s_t_minus_1:", s_t_minus_1)
        
        # 最適化問題を解く
        X_t, z_t, f_t = solve_main_(T, C, M, xi, rho, s_t_minus_1, epsilon)
        print("z_t:", z_t)

        # 収束判定
        if np.abs(f_t_minus_1 - f_t) < varepsilon:
            print("解が収束しました。")
            break

        # 更新
        f_t_minus_1 = f_t
        z_t_minus_1 = z_t
        # iteration += 1
    

        # if iteration == max_iteration:
        #     print("最大反復回数に達しました。")
        #     break

    print("最適解x:", X_t) 
    print("最適解z:", z_t)
    print("最適値f：", f_t)
    end = time.time()

In [255]:
dca_ridge(T, C, M, xi, 500, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x56ebc071
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+02]
  QObjective range [6e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

z_t: [2.61661367e-03 2.78804366e-03 2.74430881e-03 2.34957911e-11
 3.54286798e-11 1.85200876e-11 4.62966334e-11 6.16667476e-11
 5.15222992e-11 4.20086601e-11]
-----反復回数： 2 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x56ebc071
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+02]
  QObjective range [6e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 

In [256]:
dca_ridge(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xe9d3f2ec
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 2e+02]
  QObjective range [6e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

In [257]:
dca_ridge(T, C, M, xi, 100, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)



CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x818dc902
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+02]
  QObjective range [6e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.42036727e+07 -1.19603162e+07  7.98e+03 3.00e+02  1.00e+06     0s
   1   6.61282726e+06 -8.55503472e+06  2.81e+03 1.02e+02  3.66e+05     0s
   2   3.01809834e+04 -6.

0-1変換→zを使って実行可能解に持っていく

In [11]:
def subgradient(z_t_minus_1):
    # z_t_minus_1の各要素を2倍したnp.arrayを返す
    return 2 * np.array(z_t_minus_1)


# 凸最適化問題は最大Kノルムの設定に¥sum z_i <n*epsilonを追加したもの
def solv_main(T, C, M, xi, rho, epsilon, s_z_t_minus_1):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)
    # モデルの作成
    model = gp.Model("DCA")
    
    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")
    
    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z[i] for i in range(n))- gp.quicksum(z[i] * rho * s_z_t_minus_1[i] for i in range(n))
    model.setObjective(objective, sense=GRB.MINIMIZE)
    
    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])   

    model.addConstr(gp.quicksum(z[i] for i in range(n)) <= int(n * epsilon))

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])
        
    # 時間上限
    model.Params.TimeLimit = 3600
    
    # 最適化の実行
    model.optimize()
    
    # 結果の表示
    # if model.status == GRB.OPTIMAL:
    #     optimal_X = np.array([[X[k, j].x for j in range(J)] for k in range(K)]) 
    #     optimal_z = np.array([z[i].x for i in range(n)])
    #     optimal_obj_value = model.objVal
    #     return optimal_X, optimal_z, optimal_obj_value, seiyaku_time
    # else:
    #     print("最適解が見つかりませんでした。")
    #     return None, None, None, seiyaku_time

    optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
    optimal_z = np.array([z[i].x for i in range(n)])
    optimal_obj_value = model.objVal

    return optimal_X, optimal_z, optimal_obj_value

def process_vector(w, K):
    w = np.asarray(w, dtype=float)
    # print("w", w)
    # wを減少順に並べ替える
    sorted_indices = np.argsort(-np.abs(w))
    sorted_w = np.abs(w[sorted_indices])
    # print("sorted indices", sorted_indices)
    # print("sorted_w", sorted_w)

    # w(i)をK番目まで1、それ以外を0にする
    s = np.zeros_like(w)
    s[sorted_indices[:K]] = np.sign(w[sorted_indices[:K]])
    # print("s", s)

    return s

def solve_for_est(T, C, M, xi, epsilon, z_hat):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)
    # モデルの作成
    model = gp.Model("DCA")
    
    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    
    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) 
    model.setObjective(objective, sense=GRB.MINIMIZE)
    
    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z_hat[i]) * xi[i])   

    # model.addConstr(gp.quicksum(z_hat[i] for i in range(n)) <= int(n * epsilon))

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])
        
    # 時間上限
    model.Params.TimeLimit = 3600
    
    # 最適化の実行
    model.optimize()
    
    # 結果の表示
    # if model.status == GRB.OPTIMAL:
    #     optimal_X = np.array([[X[k, j].x for j in range(J)] for k in range(K)]) 
    #     optimal_z = np.array([z[i].x for i in range(n)])
    #     optimal_obj_value = model.objVal
    #     return optimal_X, optimal_z, optimal_obj_value, seiyaku_time
    # else:
    #     print("最適解が見つかりませんでした。")
    #     return None, None, None, seiyaku_time

    optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
    optimal_obj_value = model.objVal

    return optimal_X, optimal_obj_value


def dca_92_h(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon=0.001, max_iteration=100):
    # 初期値
    start = time.time()
    iteration = 0
    f_t_minus_1 = np.inf
    for iteration in range(max_iteration):
    # while True:
        print("-----反復回数：", iteration+1, "回-----")

        # 劣勾配の計算
        s_t_minus_1 = subgradient(z_t_minus_1)
        print("s_t_minus_1:", s_t_minus_1)
        
        # 最適化問題を解く
        X_t, z_t, f_t = solv_main(T, C, M, xi, rho, epsilon, s_t_minus_1)
        print("z_t:", z_t)
        f_t_c = np.sum(np.transpose(C) * X_t)
        print("最適値：",f_t_c)

        # zをK番目まで1、それ以外を0にする
        z_hat = process_vector(z_t, int(len(xi) * epsilon))
        print("z_hat:", z_hat)

        # Xを求める
        X_t_hat, f_t_hat = solve_for_est(T, C, M, xi, epsilon, z_hat)
        print("f_t_hat:", f_t_hat)

        # 収束判定
        if np.abs(f_t_minus_1 - f_t_hat) < varepsilon:
        # if np.abs(f_t- f_t_hat) < varepsilon:
        # if f_t_minus_1 < f_t_hat:
        # rho = 500で収束
        # if np.abs(f_t_minus_1 - f_t_c) < varepsilon:
            print("解が収束しました。")
            break

        # 更新
        f_t_minus_1 = f_t_hat
        # f_t_minus_1 = f_t_c
        z_t_minus_1 = z_t
        # iteration += 1
    

        # if iteration == max_iteration:
        #     print("最大反復回数に達しました。")
        #     break

    print("最適解x:", X_t_hat) 
    print("最適解z:", z_t)
    print("最適値z_01：", z_hat)
    print("最適値f：", f_t_hat)
    end = time.time()
    print("処理時間：", end-start, "秒")

In [249]:
dca_92_h(T, C, M, xi, 500, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [5e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.6923280e+00   2.660256e+02   0.000000e+00      0s
      23    1.2656361e+03   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.265636107e+03
z_t: [0.3337761  0.         0.         0.19618556 0.42065573 0.
 0.1295001  0.41993497 1.         0.40671913]
最適値： 1065.6263031942244
z_hat: [0. 0. 0. 0. 1. 0. 0. 1. 1. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 32 columns and 352 nonzeros
Model fingerprint: 0xd200d4e1

In [230]:
dca_92_h(T, C, M, xi, 1000, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.00 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

z_t: [0.         0.         0.         0.         0.         0.
 0.         0.1293242  0.12523051 0.        ]
最適値： 1508.824665057958
z_hat: [0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 32 columns and 352 nonzeros
Model fingerprint: 0xcc55e25e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-01, 1e+02]
Presolve removed 72 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 32 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.031283e+01   0.000000e+00      0s
       8    1.4898456e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.01 seconds (0.00 work units

In [203]:
dca_92(T, C, M, xi, 1000, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.01s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

In [235]:
dca_92_h(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.00 seconds (0.00 work units)
Optimal objective  9.459259375e+02


z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.43894892]
最適値： 945.9259374857263
z_hat: [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 32 columns and 352 nonzeros
Model fingerprint: 0xcbbe72ae
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-01, 1e+02]
Presolve removed 72 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 32 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.899970e+00   0.000000e+00      0s
       8    1.4166217e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.00 seconds (0.00 work units

In [205]:
dca_92(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.01s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

In [206]:
dca_92(T, C, M, xi, 100, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.00 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

In [207]:
dca_92_h(T, C, M, xi, 100, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

In [208]:
dca_92_h(T, C, M, xi, 300, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

In [213]:
dca_92(T, C, M, xi, 300, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 42 columns and 442 nonzeros
Model fingerprint: 0xa491ef02
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.01s
Presolved: 85 rows, 42 columns, 442 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.901171e+02   0.000000e+00      0s
      38    9.4592594e+02   0.000000e+00   0.000000e+00      0s

Solved in 38 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.459259375e+02
z_t: [0.41385091 0.19543927 0.05432468 0.23985252 0.39635223 0.
 0.29962996 0.48965417 0.47194735 0.4389489

DCA_ridgeも評価値に直す

In [4]:
# 凸最適化問題を解く
def solve_main_(T, C, M, xi, rho, s_z_t_minus_1, epsilon):
    '''
    K: 供給者数
    J: 需要者数
    n: シナリオ数
    '''
    K, J = C.shape
    n = len(xi)
    # モデルの作成
    model = gp.Model("DCA")
    
    # 変数の定義
    X = model.addMVar((K, J), lb=0.0, vtype=GRB.CONTINUOUS, name="X")
    z = model.addVars(n, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="z")

    # 目的関数の定義
    objective = gp.quicksum(C[k, j] * X[k, j] for k in range(K) for j in range(J)) + rho * gp.quicksum(z[i] for i in range(n))- gp.quicksum(z[i] * rho * s_z_t_minus_1[i] for i in range(n)) + 1e-6 * gp.quicksum(z[i] * z[i] for i in range(n))
    model.setObjective(objective, sense=GRB.MINIMIZE)
    
    # 制約条件の定義
    for i in range(n):
        model.addConstr(T @ X >= (1-z[i]) * xi[i])    

    # 供給制約
    for k in range(K):
        model.addConstr(gp.quicksum(X[k, j] for j in range(J)) <= M[k])
        
    # 時間上限
    model.Params.TimeLimit = 3600
    
    # 最適化の実行
    model.optimize()

    optimal_X = np.array([[X[k, j].x for k in range(K)] for j in range(J)])
    optimal_z = np.array([z[i].x for i in range(n)])
    optimal_obj_value = model.objVal

    return optimal_X, optimal_z, optimal_obj_value

def dca_ridge_h(T, C, M, xi, rho, epsilon, z_t_minus_1, varepsilon=0.001, max_iteration=100):
    # 初期値
    f_t_minus_1 = np.inf
    start = time.time()
    iteration = 0
    for iteration in range(max_iteration):
    # while True:
        print("-----反復回数：", iteration+1, "回-----")

        # 劣勾配の計算
        s_t_minus_1 = process_vector(z_t_minus_1, int(len(xi) * epsilon))
        print("s_t_minus_1:", s_t_minus_1)
        
        # 最適化問題を解く
        X_t, z_t, f_t = solve_main_(T, C, M, xi, rho, s_t_minus_1, epsilon)
        print("z_t:", z_t)

        # zをK番目まで1、それ以外を0にする
        z_hat = process_vector(z_t, int(len(xi) * epsilon))
        print("z_hat:", z_hat)

        # Xを求める
        X_t_hat, f_t_hat = solve_for_est(T, C, M, xi, epsilon, z_hat)

        # 収束判定
        if np.abs(f_t_minus_1 - f_t_hat) < varepsilon:
        # if np.abs(f_t- f_t_hat) < varepsilon:
        # if f_t_minus_1 < f_t_hat:
        # rho = 500で収束
        # if np.abs(f_t_minus_1 - f_t_c) < varepsilon:
            print("解が収束しました。")
            break

        # 更新
        f_t_minus_1 = f_t_hat
        # f_t_minus_1 = f_t_c
        z_t_minus_1 = z_t
        # iteration += 1
    

        # if iteration == max_iteration:
        #     print("最大反復回数に達しました。")
        #     break

    print("最適解x:", X_t_hat) 
    print("最適解z:", z_t)
    print("最適値z_01：", z_hat)
    print("最適値f：", f_t_hat)
    end = time.time()
    print("処理時間：", end-start, "秒")

In [9]:
dca_ridge(T, C, M, xi, 500, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-16
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x3cb19fb7
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+02]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.04s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                

In [12]:
dca_ridge_h(T, C, M, xi, 500, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3


Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x3cb19fb7
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 5e+02]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.09s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.79547309e+09 -2.64899515e+06  7.99e+03 5.24e+02  1.08e+08     0s
   1   1.15899075e+09 -2.98041436e+07  2.98e+03 1.96e+02  4.38e+07     0s
   2   7.09137928e+06 -4.66265908e+07  1.02e+0

In [13]:
dca_ridge(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x228e0486
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 2e+02]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

In [14]:
dca_ridge_h(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x228e0486
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 2e+02]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

   4   2.36616412e+04 -4.86499751e+05  2.45e-02 6.54e-03  4.24e+03     0s
   5   9.99361800e+03 -3.05850701e+04  8.90e-04 2.90e-04  3.09e+02     0s
   6   7.56783240e+03 -6.17083162e+02  0.00e+00 2.43e-05  6.05e+01     0s
   7   2.28545075e+03 -5.61009528e+02  0.00e+00 8.74e-07  2.09e+01     0s
   8   1.99398292e+03  7.89169909e+02  0.00e+00 8.50e-08  8.86e+00     0s
   9   1.67337102e+03  1.08825017e+03  0.00e+00 3.62e-08  4.30e+00     0s
  10   1.53012816e+03  1.16316631e+03  0.00e+00 1.62e-08  2.70e+00     0s
  11   1.46089270e+03  1.31394526e+03  0.00e+00 2.39e-11  1.08e+00     0s
  12   1.40263480e+03  1.38133569e+03  0.00e+00 6.08e-12  1.57e-01     0s
  13   1.39363446e+03  1.39337092e+03  0.00e+00 2.85e-11  1.94e-03     0s
  14   1.39353506e+03  1.39353476e+03  0.00e+00 1.04e-11  2.15e-06     0s
  15   1.39353495e+03  1.39353493e+03  0.00e+00 2.19e-11  1.66e-07     0s
  16   1.39353495e+03  1.39353494e+03  0.00e+00 5.88e-12  4.09e-08     0s
  17   1.39353494e+03  1.39353494e+03 

In [15]:
dca_ridge(T, C, M, xi, 100, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xa3753788
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+02]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

In [16]:
dca_ridge_h(T, C, M, xi, 100, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xa3753788
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+02]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

z_t: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
z_hat: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 32 columns and 352 nonzeros
Model fingerprint: 0xd8f6677f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+02]
Presolve removed 72 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 32 columns, 64 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.104877e+01   0.000000e+00      0s
       8    1.5941748e+03   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.594174818e+03
-----反復回数： 2 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0

In [259]:
dca_ridge(T, C, M, xi, 100, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x818dc902
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+02]
  QObjective range [6e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

In [17]:
dca_ridge_h(T, C, M, xi, 300, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x4ea90be8
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 3e+02]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

  13   1.57525967e+03  1.57525721e+03  0.00e+00 1.99e-11  1.80e-05     0s
  14   1.57525860e+03  1.57525847e+03  0.00e+00 3.59e-11  9.96e-07     0s
  15   1.57525858e+03  1.57525856e+03  0.00e+00 3.74e-11  1.47e-07     0s
  16   1.57525858e+03  1.57525858e+03  0.00e+00 2.97e-11  1.50e-08     0s
  17   1.57525858e+03  1.57525858e+03  0.00e+00 3.92e-11  2.24e-10     0s

Barrier solved model in 17 iterations and 0.01 seconds (0.00 work units)
Optimal objective 1.57525858e+03

z_t: [2.29051240e-01 4.70760581e-02 4.35524142e-02 5.64554698e-13
 2.07839595e-12 3.02329128e-13 5.87226555e-09 3.28622615e-01
 1.25230513e-01 4.43910108e-12]
z_hat: [1. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 32 columns and 352 nonzeros
Model fingerprint: 0xd062e77e
Coefficien

In [18]:
dca_ridge(T, C, M, xi, 300, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x4ea90be8
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 3e+02]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

   3   8.96986408e+04 -8.81873308e+06  1.02e-01 1.31e-01  7.86e+04     0s
   4   3.07318915e+04 -7.33280852e+05  2.44e-02 6.53e-03  6.35e+03     0s
   5   1.01849668e+04 -4.86776392e+04  8.57e-04 2.82e-04  4.49e+02     0s
   6   8.46487995e+03 -8.20978253e+02  0.00e+00 1.68e-05  6.85e+01     0s
   7   2.44362497e+03 -1.73115491e+02  0.00e+00 1.21e-06  1.93e+01     0s
   8   2.12500268e+03  1.01822436e+03  0.00e+00 6.57e-08  8.14e+00     0s
   9   1.76459694e+03  1.41455785e+03  0.00e+00 8.75e-09  2.57e+00     0s
  10   1.62705388e+03  1.51842231e+03  0.00e+00 2.66e-09  7.99e-01     0s
  11   1.58449992e+03  1.55468114e+03  0.00e+00 8.40e-11  2.19e-01     0s
  12   1.57544591e+03  1.57480271e+03  0.00e+00 2.99e-11  4.73e-03     0s
  13   1.57525967e+03  1.57525721e+03  0.00e+00 1.99e-11  1.80e-05     0s
  14   1.57525860e+03  1.57525847e+03  0.00e+00 3.59e-11  9.96e-07     0s
  15   1.57525858e+03  1.57525856e+03  0.00e+00 3.74e-11  1.47e-07     0s
  16   1.57525858e+03  1.57525858e+03 

In [262]:
dca_ridge_h(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xe9d3f2ec
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 2e+02]
  QObjective range [6e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.01s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

In [263]:
dca_ridge(T, C, M, xi, 200, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0xe9d3f2ec
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 2e+02]
  QObjective range [6e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

In [19]:
dca_ridge_h(T, C, M, xi, 1000, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)



CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x61474187
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+03]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.58743767e+09 -2.68641763e+06  7.99e+03 5.24e+02  2.16e+08     0s
   1   2.31699997e+09 -5.84282945e+07  2.98e+03 1.96e+02  8.75e+07     0s
   2   1.39712743e+07 -9.3

In [20]:
dca_ridge(T, C, M, xi, 1000, 0.3, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x61474187
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+03]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 


z_t: [1.01831026e-01 9.44525498e-02 9.24097687e-02 1.54304928e-12
 1.81207687e-12 1.37966354e-12 1.97090409e-12 2.12202797e-12
 2.02972599e-12 1.91505082e-12]
-----反復回数： 2 回-----
s_t_minus_1: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 84 rows, 42 columns and 432 nonzeros
Model fingerprint: 0x61474187
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e-01, 1e+01]
  Objective range  [1e+01, 1e+03]
  QObjective range [2e-06, 2e-06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+02]
Presolve time: 0.00s
Presolved: 84 rows, 42 columns, 432 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.600e+02
 Factor NZ  : 2.754e+03
 Factor Ops : 1.092e+05

In [267]:
dca_92_h(T, C, M, xi, 500, 0.1, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.

In [269]:
dca_92(T, C, M, xi, 500, 0.1, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.

In [272]:
dca_92_h(T, C, M, xi, 100, 0.1, z_t_minus_1)

-----反復回数： 1 回-----
s_t_minus_1: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.